# Quantum Hearing
The goal of this project is to design an alternative way for anyone interested in quantum computing to experience quantum state evolution through music.

In [38]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, BasicAer, execute
from qiskit.visualization import plot_bloch_multivector
from ipywidgets import widgets
import math

In [2]:
def get_unitary(circuit):
    backend = BasicAer.get_backend("unitary_simulator")
    job_sim = execute(circuit, backend)
    result_sim = job_sim.result()
    output_state = result_sim.get_unitary(circuit, decimals=3)
    return output_state

def get_statevector(circuit):
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circuit, backend)
    return job.result().get_statevector(circuit)

In [3]:
qasm = widgets.Text()

input_text = widgets.Text(
 placeholder='Your QASM here',
 description='Raw QASM:'
 )

def bind_input_to_output(sender):
    qasm.value = input_text.value

input_text.on_submit(bind_input_to_output)

## QASM
Insert QASM code below and hit enter to save input

In [27]:
input_text
# Insert QASM code below and hit enter to save input

Text(value='OPENQASM 2.0; include "qelib1.inc";  qreg q[1]; creg c[1];  h q[0]; s q[0];', description='Raw QAS…

In [32]:
try:
    circuit = QuantumCircuit.from_qasm_str(qasm.value)
    vector = get_statevector(circuit)
    unitary = get_unitary(circuit)
    display(circuit.draw())
    num_qubits = circuit.num_qubits
    if num_qubits >= 2:
        display(plot_bloch_multivector(vector))
except:
    print("Either no circuit was input or you forgot to hit enter \U0001f600")

┌───┐┌───┐
q_0: ┤ H ├┤ S ├
     └───┘└───┘
c: 1/══════════

In [39]:
lines = qasm.value.split(';')[4:-1]
gates = []
qubit = []
gate_phases = {"s":(math.pi/2), "sdg":(-math.pi/2), "t":(math.pi/4), "tdg":(-math.pi/4)}
for i,line in enumerate(lines):
    lines[i] = line.strip()
    gates.append(lines[i].split(' ')[0])
    qubit.append(lines[i].split(' ')[1][2])
print(gates)
print(qubit)

['h', 's']
['0', '0']


In [35]:
print(unitary)
print(vector)

[[ 7.07106781e-01+0.00000000e+00j  7.07106781e-01-8.65956056e-17j]
 [ 4.32978028e-17+7.07106781e-01j -1.29893408e-16-7.07106781e-01j]]
[7.07106781e-01+0.j         4.32978028e-17+0.70710678j]
